In [ ]:
#changing the working directory

%cd /content/drive/MyDrive/Kaggle/Forest-Cover-Type-Prediction/

/content/drive/MyDrive/Kaggle/Forest-Cover-Type-Prediction


In [ ]:
# imports

from sklearn.decomposition import PCA

import pandas as pd
import tensorflow as tf


In [ ]:
# preprocessing methods

soils_cols = ['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40']
area_cols = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4']

In [ ]:
def preprocess_forest_data(df):
  def get_soil(row):
    for c in soils_df.columns:
      if row[c]==1:
        return int(c.replace("Soil_Type", ""))
  def get_area(row):
    for c in area_df.columns:
      if row[c]==1:
        return int(c.replace("Wilderness_Area", ""))

  print("Compressing soils data...")
  soils_df = df[soils_cols]
  soils_series = soils_df.apply(get_soil, axis=1)
  df['Soil_Type'] = soils_series
  df = df.drop(soils_cols, axis=1)

  print("Compressing area data...")
  area_df = df[area_cols]
  area_series = area_df.apply(get_area, axis=1)
  df['Wilderness_Area'] = area_series
  df = df.drop(area_cols, axis=1)

  # extract & drop IDs
  ids = df['Id']
  df = df.drop(columns=['Id'])

  print("Adding PCA features...")
  df_scaled = (df- df.mean(axis=0)) / df.std(axis=0)
  pca = PCA()
  df_pca = pca.fit_transform(df_scaled)
  component_names = [f"PC{i+1}" for i in range(df_pca.shape[1])]
  df_pca = pd.DataFrame(df_pca, columns=component_names)
  df["PC1"] = df_pca["PC1"]
  df["PC5"] = df_pca["PC5"]

  # drop low information columns
  df = df.drop(["Vertical_Distance_To_Hydrology", "Aspect", "Slope", "Hillshade_3pm", "Hillshade_Noon"], axis=1)
  return df

## Process train set

In [ ]:
train_data_path = "train.csv"

forest_data = pd.read_csv(train_data_path)
forest_data

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,15116,2607,243,23,258,7,660,170,251,214,1282,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15116,15117,2603,121,19,633,195,618,249,221,91,1325,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15117,15118,2492,134,25,365,117,335,250,220,83,1187,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15118,15119,2487,167,28,218,101,242,229,237,119,932,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
X = forest_data.copy()
y = X.pop("Cover_Type")
X = preprocess_forest_data(X)
X

Compressing soils data...
Compressing area data...
Adding PCA features...


,Elevation,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area,PC1,PC5
0,2596,258,510,221,6279,29,1,2.477160,2.349859
1,2590,212,390,220,6225,29,1,2.443291,2.264236
2,2804,268,3180,234,6121,12,1,2.788295,3.583020
3,2785,242,3090,238,6211,30,1,3.022746,3.116338
4,2595,153,391,220,6172,29,1,2.358213,2.230608
...,...,...,...,...,...,...,...,...,...
15115,2607,258,660,170,1282,4,3,-0.735746,0.259090
15116,2603,633,618,249,1325,4,3,-0.830528,0.725291
15117,2492,365,335,250,1187,4,3,-1.613414,0.500129
15118,2487,218,242,229,932,4,3,-1.698264,0.286242


## Process submission test set

In [ ]:
test_data_path = "test.csv"
test_set = pd.read_csv(test_data_path)
test_set

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,6645,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,6675,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,6344,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,6374,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,6404,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565887,581008,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565888,581009,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565889,581010,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565890,581011,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
processed_test_set = preprocess_forest_data(test_set)
processed_test_set

Compressing soils data...
Compressing area data...
Adding PCA features...


,Elevation,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area,PC1,PC5
0,2680,0,2684,196,6645,29,1,-0.581266,1.445021
1,2683,0,2654,201,6675,29,1,-1.585880,1.647809
2,2713,0,2980,206,6344,29,1,-1.619453,1.448532
3,2709,0,2950,208,6374,29,1,-1.579659,1.407863
4,2706,0,2920,210,6404,29,1,-1.540727,1.373304
...,...,...,...,...,...,...,...,...,...
565887,2396,85,108,240,837,2,3,2.130164,0.702718
565888,2391,67,95,240,845,2,3,2.089216,0.663545
565889,2386,60,90,236,854,2,3,2.121597,0.670026
565890,2384,60,90,230,864,2,3,2.205713,0.701379


## Let the training begin

In [ ]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2
X = X.drop(["PC1", "PC5"], axis=1)

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X)
normalizer(X.iloc[:3])

<tf.Tensor: shape=(3, 7), dtype=float32, numpy=
array([[-0.36709452,  0.14663921, -0.90868014,  0.27145085,  4.3348055 ,
         0.77841055, -1.6077906 ],
       [-0.38146013, -0.07233712, -0.99924463,  0.23872861,  4.28571   ,
         0.77841055, -1.6077906 ],
       [ 0.13091312,  0.19424278,  1.10638   ,  0.69683987,  4.191156  ,
        -0.5679593 , -1.6077906 ]], dtype=float32)>

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_basic_model()
model.fit(X, y, epochs=15, batch_size=BATCH_SIZE)